In [1]:
!pip install duckduckgo-search python-pptx ollama


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 1.1 MB/s  0:00:01 eta 0:00:010m
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.12
    Uninstalling pydantic-1.10.12:
      Successfully uninstalled pydantic-1.10.12
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [ollama]2m3/5 [pydantic]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-cloud-auth 0.1.4 requires pydantic<2.0, but you have pydantic 2.11.7 which is incompatible.


In [9]:
!pip uninstall -y duckduckgo_search
!pip install ddgs


Found existing installation: duckduckgo_search 8.1.1
Uninstalling duckduckgo_search-8.1.1:
  Successfully uninstalled duckduckgo_search-8.1.1
  Using cached lxml-6.0.1-cp311-cp311-macosx_10_9_universal2.whl.metadata (3.8 kB)
Using cached lxml-6.0.1-cp311-cp311-macosx_10_9_universal2.whl (8.4 MB)
  Attempting uninstall: lxml
    Found existing installation: lxml 5.4.0
    Uninstalling lxml-5.4.0:
      Successfully uninstalled lxml-5.4.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [ddgs]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
htmldate 1.9.3 requires lxml<6,>=5.3.0; platform_system != "Darwin" or python_version > "3.8", but you have lxml 6.0.1 which is incompatible.


In [1]:
from ddgs import DDGS

def get_web_context(topic, num_results=3):
    ddgs = DDGS()
    results = []
    for r in ddgs.text(topic, max_results=num_results):
        results.append(r['body'])
    return "\n\n".join(results)



In [2]:
import ollama

def generate_slides_with_ollama(topic, web_context, model_name='llama2'):
    prompt = f'''
You are a presentation assistant.
Create a PowerPoint slide deck structure with these slides:
  1. Title
  2. Overview
  3-4. Key Points/Trends/Arguments (1 slide per point)
  7. Conclusion or Key Takeaways
For each slide output a JSON object: {{"title": ..., "bullets": [ ... ]}}
Each bullet should be concise (max 12 words).
Base your answer on the following recent web content:
WEB CONTEXT:
{web_context}
TOPIC: {topic}
Output a Python list of dicts (one per slide).
'''
    # Use Ollama's python client, which calls the local model
    response = ollama.generate(model=model_name, prompt=prompt)
    # The model may output non-executable code, clean it up:
    slides = None
    try:
        import ast
        slides = ast.literal_eval(response['response'].strip())
    except Exception:
        print("LLM output failed to parse, here is the raw result:\n", response['response'])
    return slides


In [3]:
from pptx import Presentation

def create_pptx(slides, filename):
    prs = Presentation()
    for idx, slide_content in enumerate(slides):
        if idx == 0:
            layout = prs.slide_layouts  # title slide
        else:
            layout = prs.slide_layouts[12]  # title + bullets
        slide = prs.slides.add_slide(layout)
        slide.shapes.title.text = slide_content['title']
        if len(slide.placeholders) > 1 and slide_content.get('bullets'):
            body_shape = slide.placeholders[12]
            tf = body_shape.text_frame
            tf.clear()
            for bullet in slide_content['bullets']:
                p = tf.add_paragraph()
                p.text = bullet
    prs.save(filename)


In [4]:
# 1. Set your topic
topic = "Pollution"

# 2. Fetch recent web content for the topic
web_context = get_web_context(topic, num_results=5)
print("Web Context Fetched:\n", web_context)
short_context = web_context[:500]  # take just first 2000 chars

# 3. Generate slides using your local Ollama LLM
slides = generate_slides_with_ollama(topic, short_context, model_name='llama2')

# 4. If slides were successfully generated, save to pptx
if slides:
    pptx_filename = topic.replace(" ", "_") + ".pptx"
    create_pptx(slides, pptx_filename)
    print(f"PowerPoint presentation saved as: {pptx_filename}")
else:
    print("Slide parsing from LLM failed. Check output above.")


Web Context Fetched:
 Pollution is the introduction of contaminants into the natural environment that cause harm. Pollution can take the form of any substance (solid, liquid, or gas) or energy (such as radioactivity, heat, sound, or light). Pollutants, the components of pollution, can be either foreign substances/energies or naturally occurring contaminants.Although environmental pollution can be caused by natural events, the word pollution generally implies that the contaminants have a human source, such as manufacturing, extractive industries, poor waste management, transportation or agriculture. Pollution is often classed as point source (coming from a highly concentrated specific site, such as a factory, mine, construction site), or nonpoint source pollution (coming from a widespread distributed sources, such as microplastics or agricultural runoff).Many sources of pollution were unregulated parts of industrialization during the 19th and 20th centuries until the emergence of enviro

KeyboardInterrupt: 

In [17]:
response = ollama.generate(model="llama2", prompt="Hello, please say hi.")
print(response['response'])



Hi there! *waves* How are you?
